In [2]:
from tensorflow.keras.layers import Layer
import tensorflow as tf
import tensorflow.keras as keras
from keras.models import Model
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import AveragePooling2D, AveragePooling1D
from keras.layers.pooling import GlobalAveragePooling2D,GlobalAveragePooling1D
from keras.layers import Input, merge, Flatten,Reshape
import scipy.io as sio
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

import tensorflow.python.keras.engine

from keras.layers.recurrent import LSTM
from keras.layers import Bidirectional
from keras.regularizers import l2
import keras.backend as K

from tensorflow.keras.layers import Layer
from keras.layers import Conv1D,Conv2D, MaxPooling2D,concatenate
from tensorflow.keras.layers import * 
from tensorflow.keras.layers import Layer
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.optimizers import Adam,SGD
from cProfile import label
import csv
import numpy as np
import tensorflow.keras.utils as kutils
from tensorflow.python.keras.initializers import get
from tensorflow import keras


from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, recall_score, accuracy_score, precision_score


2023-07-13 20:18:29.803375: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-13 20:18:29.803553: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-13 20:18:29.803675: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-13 20:18:29.803836: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-13 20:18:29.803954: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

In [3]:
def getMatrixLabel(positive_position_file_name,sites, window_size=49  , empty_aa = '*'):
    prot = []  # list of protein name
    pos = []  # list of position with protein name
    rawseq = []
    all_label = []

    short_seqs = []
    half_len = (window_size - 1) / 2

    with open(positive_position_file_name, 'r') as rf:
        reader = csv.reader(rf)
            
        for row in reader:
            # print("row[0]:",int(row[0]))
            position = int(row[2])
            sseq = row[3]
            rawseq.append(row[3])
            center = sseq[position - 1]
            if center in sites:
                all_label.append(int(row[0]))
                # print("length of all_label",len(all_label))
                prot.append(row[1])
                pos.append(row[2])

                
                if position - half_len > 0:
                    start = position - half_len
                    start = int(start)
                    position = int(position)
                    left_seq = sseq[start - 1:position - 1]
                else:
                    #如果在左侧,left_seq中最右侧为位点
                    left_seq = sseq[0:position - 1]

                end = len(sseq)
                if position + half_len < end:
                    end = position + half_len
                    end = int(end)
                right_seq = sseq[position:end]

                if len(left_seq) < half_len:
                    nb_lack = half_len - len(left_seq)
                    nb_lack = int(nb_lack)
                    left_seq = ''.join([empty_aa for count in range(nb_lack)]) + left_seq

                if len(right_seq) < half_len:
                    nb_lack = half_len - len(right_seq)
                    nb_lack = int(nb_lack)
                    right_seq = right_seq + ''.join([empty_aa for count in range(nb_lack)])
                shortseq = left_seq + center + right_seq
                short_seqs.append(shortseq)
        targetY = kutils.to_categorical(all_label)
        letterDict = {}
        letterDict["A"] = 0
        letterDict["C"] = 1
        letterDict["D"] = 2
        letterDict["E"] = 3
        letterDict["F"] = 4
        letterDict["G"] = 5
        letterDict["H"] = 6
        letterDict["I"] = 7
        letterDict["K"] = 8
        letterDict["L"] = 9
        letterDict["M"] = 10
        letterDict["N"] = 11
        letterDict["P"] = 12
        letterDict["Q"] = 13
        letterDict["R"] = 14
        letterDict["S"] = 15
        letterDict["T"] = 16
        letterDict["V"] = 17
        letterDict["W"] = 18
        letterDict["Y"] = 19
        letterDict["*"] = 20
        # letterDict["?"] = 21
        Matr = np.zeros((len(short_seqs), window_size))
        samplenumber = 0
        for seq in short_seqs:
            AANo = 0
            for AA in seq:
                if AA not in  letterDict:
                    # AANo += 1
                    continue
                Matr[samplenumber][AANo] = letterDict[AA]
                AANo = AANo+1
            samplenumber = samplenumber + 1
    # print('data process finished')
    print("matr.shape",Matr.shape)
    return Matr, targetY ,all_label

In [4]:
import csv
train_file_name="/storage/yq/DephosSite/dephos_data/train_Y .csv"
max_features=21
win1=31
win2=51
x_train1,y_train,z1 = getMatrixLabel(train_file_name, ('Y'), win1)
x_train2, _,z1 = getMatrixLabel(train_file_name, ('Y'), win2)
perm = np.random.permutation(len(x_train1))
print(perm)
x_train1 = x_train1[perm]
x_train2 = x_train2[perm]
y_train = y_train[perm]
# z1 = z1[perm]
print(y_train[1:10])

matr.shape (201, 31)
matr.shape (201, 51)
[ 22 149  86  72  79 127  16  50 102  89 151 179 198 109   4  65 129  94
  38 121 134  17 124  60 192  97  43 197   0 114   8  28 181 153 143  53
  14  11  49   6  20 135 172 178  48  25 189  18 199  15  41  99  24  92
  80 105  30  64 120  37  26 174  95 133  35  51 141 157 103  66  87  62
 180 110 137  12  96 186 108  42  45 113 139  27 158   2 122  13 177  57
 191  67 159  33 185  46 170  76  71  56   9 167 118 130   7  73  19  32
 140 161 116 165  63 106 125 138 123 164 166 156  21 187  70  93 136 196
 101 147 126  81 148  75  39 132 146  98 183  90 200  85  91 169 145  84
 117 119 173  83 111  34 128 142  40 171  52 184  74 175  82  88 131 190
 115 100  47 194  29 195 154 144 152  36 150   1 112  31 160   5  54 193
 168  69  23  68 104  77 176   3 155 182 163  59  55 107 188  61  78  58
 162  44  10]
[[1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]]


In [5]:
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
tensorboard = TensorBoard(log_dir='log')
print(tensorboard)
checkpoint = ModelCheckpoint(filepath='/storage/yq/DephosSite/model_multi_713_y_retrainST.h5',monitor='val_loss',mode='min' ,save_best_only='True')
# ROnPlateau = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=30, verbose=0, mode='auto', cooldown=0, min_lr=0)
early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=10, verbose=1)
# ROnPlateau = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=30, verbose=0, mode='auto', cooldown=0, min_lr=0)

# 将所有回调函数放入一个列表中
callback_lists = [tensorboard, checkpoint]
# 加载已经训练好的模型
model1 = load_model("/storage/yq/DephosSite/model/mulit_31_51_hn.h5")

# 修改输出层
# 解冻所有层
for layer in model1.layers:
    layer.trainable = True

    
optimizer= Adam(learning_rate=0.000001)
# 编译模型
model1.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# 训练模型
model1.fit([x_train1,x_train2],y_train,batch_size=16,epochs=200,validation_split=0.2,callbacks=callback_lists)

/opt/anaconda3/envs/zzqp/lib/python3.8/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/200


2023-07-13 20:18:36.962902: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8401
2023-07-13 20:18:37.861349: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-07-13 20:18:37.862197: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.6
2023-07-13 20:18:37.862208: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2023-07-13 20:18:37.862240: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be o

10/10 [==============================] - 7s 142ms/step - loss: 0.8149 - accuracy: 0.5938 - val_loss: 1.3217 - val_accuracy: 0.5122
Epoch 2/200
10/10 [==============================] - 1s 71ms/step - loss: 0.8354 - accuracy: 0.6125 - val_loss: 1.2950 - val_accuracy: 0.5122
Epoch 3/200
10/10 [==============================] - 1s 72ms/step - loss: 0.7968 - accuracy: 0.6313 - val_loss: 1.2706 - val_accuracy: 0.5122
Epoch 4/200
10/10 [==============================] - 1s 68ms/step - loss: 0.8643 - accuracy: 0.5938 - val_loss: 1.2491 - val_accuracy: 0.5122
Epoch 5/200
10/10 [==============================] - 1s 79ms/step - loss: 0.8094 - accuracy: 0.6187 - val_loss: 1.2297 - val_accuracy: 0.5122
Epoch 6/200
10/10 [==============================] - 1s 78ms/step - loss: 0.7878 - accuracy: 0.6125 - val_loss: 1.2120 - val_accuracy: 0.4878
Epoch 7/200
10/10 [==============================] - 1s 78ms/step - loss: 0.7894 - accuracy: 0.6438 - val_loss: 1.1945 - val_accuracy: 0.4878
Epoch 8/200
10/10

10/10 [==============================] - 1s 74ms/step - loss: 0.5307 - accuracy: 0.7437 - val_loss: 0.8566 - val_accuracy: 0.5610
Epoch 59/200
10/10 [==============================] - 1s 72ms/step - loss: 0.5978 - accuracy: 0.6875 - val_loss: 0.8539 - val_accuracy: 0.5610
Epoch 60/200
10/10 [==============================] - 1s 79ms/step - loss: 0.6116 - accuracy: 0.6875 - val_loss: 0.8502 - val_accuracy: 0.5610
Epoch 61/200
10/10 [==============================] - 1s 73ms/step - loss: 0.5680 - accuracy: 0.7375 - val_loss: 0.8481 - val_accuracy: 0.5854
Epoch 62/200
10/10 [==============================] - 1s 80ms/step - loss: 0.5813 - accuracy: 0.7125 - val_loss: 0.8455 - val_accuracy: 0.5854
Epoch 63/200
10/10 [==============================] - 1s 71ms/step - loss: 0.5149 - accuracy: 0.7500 - val_loss: 0.8431 - val_accuracy: 0.5854
Epoch 64/200
10/10 [==============================] - 1s 71ms/step - loss: 0.5453 - accuracy: 0.7250 - val_loss: 0.8407 - val_accuracy: 0.5854
Epoch 65/200

10/10 [==============================] - 1s 76ms/step - loss: 0.5296 - accuracy: 0.7563 - val_loss: 0.7731 - val_accuracy: 0.6341
Epoch 116/200
10/10 [==============================] - 1s 79ms/step - loss: 0.4764 - accuracy: 0.7875 - val_loss: 0.7725 - val_accuracy: 0.6341
Epoch 117/200
10/10 [==============================] - 1s 74ms/step - loss: 0.4777 - accuracy: 0.8000 - val_loss: 0.7719 - val_accuracy: 0.6341
Epoch 118/200
10/10 [==============================] - 1s 76ms/step - loss: 0.4569 - accuracy: 0.8125 - val_loss: 0.7716 - val_accuracy: 0.6341
Epoch 119/200
10/10 [==============================] - 1s 76ms/step - loss: 0.4449 - accuracy: 0.8062 - val_loss: 0.7715 - val_accuracy: 0.6341
Epoch 120/200
10/10 [==============================] - 1s 79ms/step - loss: 0.4757 - accuracy: 0.7750 - val_loss: 0.7711 - val_accuracy: 0.6341
Epoch 121/200
10/10 [==============================] - 1s 87ms/step - loss: 0.4741 - accuracy: 0.7625 - val_loss: 0.7708 - val_accuracy: 0.6341
Epoch 

Epoch 172/200
10/10 [==============================] - 0s 13ms/step - loss: 0.4318 - accuracy: 0.8375 - val_loss: 0.7592 - val_accuracy: 0.6341
Epoch 173/200
10/10 [==============================] - 0s 13ms/step - loss: 0.4406 - accuracy: 0.8250 - val_loss: 0.7594 - val_accuracy: 0.6341
Epoch 174/200
10/10 [==============================] - 0s 13ms/step - loss: 0.4640 - accuracy: 0.7437 - val_loss: 0.7591 - val_accuracy: 0.6341
Epoch 175/200
10/10 [==============================] - 0s 13ms/step - loss: 0.4207 - accuracy: 0.7812 - val_loss: 0.7592 - val_accuracy: 0.6341
Epoch 176/200
10/10 [==============================] - 0s 13ms/step - loss: 0.4666 - accuracy: 0.7750 - val_loss: 0.7590 - val_accuracy: 0.6341
Epoch 177/200
10/10 [==============================] - 0s 13ms/step - loss: 0.4134 - accuracy: 0.8125 - val_loss: 0.7591 - val_accuracy: 0.6341
Epoch 178/200
10/10 [==============================] - 0s 13ms/step - loss: 0.4228 - accuracy: 0.7937 - val_loss: 0.7591 - val_accuracy:

In [6]:
test_file_name="/storage/yq/DephosSite/dephos_data/test_Y.csv"
win1=31
win2=51
win3=71
[x_test1,y_test,z] = getMatrixLabel(test_file_name, ('Y'), win1)
[x_test2,_,_] = getMatrixLabel(test_file_name, ('Y'), win2)
[x_test3,_,_] = getMatrixLabel(test_file_name, ('Y'), win3)
perm = np.random.permutation(len(x_test1))
x_test1 = x_test1[perm]
x_test2 = x_test2[perm]
y_test = y_test[perm]

matr.shape (50, 31)
matr.shape (50, 51)
matr.shape (50, 71)


In [7]:
def calc(TN, FP, FN, TP):
    SN = TP / (TP + FN)  # recall
    SP = TN / (TN + FP)
    # Precision = TP / (TP + FP)
    ACC = (TP + TN) / (TP + TN + FN + FP)
    F1 = (2 * TP) / (2 * TP + FP + FN)
    fz = TP * TN - FP * FN
    fm = (TP + FN) * (TP + FP) * (TN + FP) * (TN + FN)
    MCC = fz / pow(fm, 0.5)
    PRE=TP/(TP+FP)
    return SN, SP, ACC, MCC,F1,PRE

In [8]:
y_pre_test = model1.predict([x_test1,x_test2])
tn, fp, fn, tp  = confusion_matrix(y_test.argmax(axis=1),y_pre_test.argmax(axis=1)).ravel()
sn, sp, acc, mcc,f1,Pre= calc(tn, fp, fn, tp)
from sklearn.metrics import roc_auc_score
AUC=roc_auc_score(y_test[:,1], y_pre_test[:,1])
sn, sp, acc, mcc,f1,Pre,AUC

(0.9166666666666666,
 0.8076923076923077,
 0.86,
 0.726106525690946,
 0.8627450980392157,
 0.8148148148148148,
 0.9519230769230769)